In [ ]:
import matplotlib.pyplot as plt
import os
os.environ.update(dict(CUDA_VISIBLE_DEVICES='3'))

import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from tqdm import tqdm

In [ ]:
# data = torch.load('/ssd1/tta/imagenet_val_resnet50_lyrfts_full.pth')
data = torch.load('/ssd1/tta/imagenet_val_resnet50_shf_bn_full.pth')

In [ ]:
#features, labels
features_inc = data['features']
features_im = data['ifeatures'] # 50000, C (2048, 1024, 512, 256)
labels = data['labels']
logits_inc = data['logits']
logits_im = data['ilogits']
correct = data['correct'].type(torch.int)

layer = -1

def cidx(i):
    return labels == i

def cov(tensor: torch.Tensor, rowvar=True, bias=False):
    """Estimate a covariance matrix (np.cov)"""
    tensor = tensor if rowvar else tensor.transpose(-1, -2)
    tensor = tensor - tensor.mean(dim=-1, keepdim=True)
    factor = 1 / (tensor.shape[-1] - int(not bool(bias)))
    return factor * tensor @ tensor.transpose(-1, -2).conj()

def cls_distrib(c: int):
    ft: torch.Tensor = features_im[layer][cidx(c)].cuda() # 50, C
    cv = cov(ft, rowvar=False).type(torch.float64)
    L = torch.linalg.cholesky(cv + torch.eye(ft.shape[1]).cuda() * 1e-6)
    return ft.mean(0).cpu(), torch.cholesky_inverse(L).cpu() #cov(ft, rowvar=False) #ft.var(0).sqrt()

# cls_distribs = []
# for c in tqdm(range(1000)):
#     cls_distribs.append(cls_distrib(c))